In [1]:
# Importació de llibreries per a modelatge, processament de text i validació

import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier  # Classificador Gradient Boosting optimitzat

# Càrrega del conjunt de dades '20 Newsgroups', una col·lecció de textos en anglès classificats per temes

dades = fetch_20newsgroups(subset='all', remove=())  # Es carrega el conjunt complet, sense eliminar cap part

# Es crea un DataFrame amb el text dels documents i la seva etiqueta numèrica
df = pd.DataFrame({
    'text': dades.data,
    'etiqueta_original': dades.target
})

# S’afegeix una columna amb el nom de la categoria textual corresponent a cada etiqueta
noms_categories = dades.target_names
df['tema'] = df['etiqueta_original'].apply(lambda x: noms_categories[x])

# Definició d'una variable binària "esport"
# Classes positives: categories relacionades amb esports i motors

categories_esport = ['rec.sport.baseball', 'rec.sport.hockey', 'rec.autos', 'rec.motorcycles']
df['esport'] = df['tema'].apply(lambda x: 1 if x in categories_esport else 0)

# Separació de les dades (X) i etiquetes binàries (y)

X = df['text'].values
y = df['esport'].values

# Definició d’una llista de paraules buides personalitzada
# Aquestes paraules seran ignorades durant la vectorització

stopwords = [
    'as', 'an', 'the', 'in', 'on', 'at', 'to', 'of', 'and', 'or',
    'is', 'it', 'for', 'with', 'that', 'this', 'was', 'be',
    'are', 'were', 'been', 'from', 'by', 'about', 'into', 'out',
    'up', 'down', 'over', 'under', 'then', 'than', 'so', 'but', 'not'
]


In [2]:
# Inicialització de la validació creuada 
# Es divideix el conjunt de dades en 5 particions (folds)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []  # Llista per emmagatzemar la puntuació AUC de cada fold
# Entrenament i avaluació del model per a cada fold

for train_idx, test_idx in cv.split(X, y):
    # Separació de les dades en conjunt d'entrenament i de prova
    X_train_raw, X_test_raw = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Vectorització del text amb TF-IDF
    # El vectoritzador només s’ajusta sobre el conjunt d'entrenament
    # per evitar qualsevol filtració d'informació (data leakage)

    vectoritzador = TfidfVectorizer(max_features=10000, stop_words=stopwords)
    X_train = vectoritzador.fit_transform(X_train_raw)
    X_test = vectoritzador.transform(X_test_raw)


    # Entrenament del model XGBoost (Gradient Boosting per classificació binària)
    # Paràmetres comuns: profunditat màxima, nombre d'arbres, mètrica i semilla


    model = XGBClassifier(
        max_depth=6,
        n_estimators=500,  
        use_label_encoder=False, # Es desactiva per evitar warnings antics
        eval_metric='logloss',
        random_state=42
    )
    model.fit(X_train, y_train)

    print("iteracio")  # Missatge indicatiu de progrés

    # Predicció de probabilitats per a la classe positiva (1)
    # i càlcul de la mètrica AUC-ROC sobre el conjunt de prova

    probs = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, probs)
    scores.append(auc)  # S’afegeix el resultat del fold actual

    # Impressió de les puntuacions acumulades fins al moment
    print(scores)

# Resultats finals de la validació creuada

scores = np.array(scores)

# Impressió de les puntuacions AUC per a cada fold amb 4 decimals
print("Puntuacions AUC-ROC per fold:", np.round(scores, 4))

# Impressió de la mitjana global de l’AUC-ROC
print(f"Mitjana AUC-ROC (CV): {scores.mean():.4f}")


iteracio
[np.float64(0.9914700435601005)]


iteracio
[np.float64(0.9914700435601005), np.float64(0.9903283656763467)]


iteracio
[np.float64(0.9914700435601005), np.float64(0.9903283656763467), np.float64(0.9891287077837895)]


iteracio
[np.float64(0.9914700435601005), np.float64(0.9903283656763467), np.float64(0.9891287077837895), np.float64(0.9942759542752444)]


iteracio
[np.float64(0.9914700435601005), np.float64(0.9903283656763467), np.float64(0.9891287077837895), np.float64(0.9942759542752444), np.float64(0.9922045568934963)]
Puntuacions AUC-ROC per fold: [0.9915 0.9903 0.9891 0.9943 0.9922]
Mitjana AUC-ROC (CV): 0.9915
